In [65]:
# !pip install timm
# !pip install interpretdl
# !pip install x2paddle
# !pip install paddlepaddle

In [67]:
import torch
import timm
from PIL import Image
from torchvision import transforms
from interpretdl import GradCAMInterpreter
from x2paddle.convert import pytorch2paddle

In [110]:
model = timm.create_model("tiny_vit_5m_224", pretrained=False, num_classes=37)
state_dict = torch.load("tinyvit_trained_model.pth")
print("About to load state dict...")
try:
    model.load_state_dict(state_dict)
    print("State dict loaded successfully!")
except Exception as e:
    print(f"Error while loading state dict: {e}")

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)
_ = model.eval()

About to load state dict...
State dict loaded successfully!


In [ ]:
scripted_model = torch.jit.script(model)
scripted_model.save("scripted_model.pt")

In [102]:
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),  # Resize all images to 224x224
#     transforms.ToTensor(),          # Convert to tensor
#     transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize
# ])

# image = Image.open("cat.jpeg")
# input_tensor = transform(image).unsqueeze(0)

input_examples = torch.randn(1, 3, 224, 224)  # Start with 3D tensor
# input_examples = input_examples.unsqueeze(0)  # Add batch dimension explicitly
print("Input shape:", input_examples.size())  # Should print torch.Size([1, 3, 224, 224])

Input shape: torch.Size([4, 3, 224, 224])


In [107]:
pytorch2paddle(
    module=scripted_model,
    save_dir="paddle_model_dir",
    jit_type="script"
)

Now translating model from PyTorch to Paddle.


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_342/2318951956.py", line 1, in <module>
    pytorch2paddle(module=model, save_dir="converted_paddle_model", jit_type="script")
  File "/home/jovyan/.local/lib/python3.8/site-packages/x2paddle/convert.py", line 398, in pytorch2paddle
    model = ScriptDecoder(module, input_examples)
  File "/home/jovyan/.local/lib/python3.8/site-packages/x2paddle/decoder/pytorch_decoder.py", line 47, in __init__
    self.script = torch.jit.script(module)
  File "/home/jovyan/.local/lib/python3.8/site-packages/torch/jit/_script.py", line 1338, in script
    if maybe_already_compiled_fn:
  File "/home/jovyan/.local/lib/python3.8/site-packages/torch/jit/_recursive.py", line 558, in create_script_module
    if isinstance(nn_module, (torch.nn.ModuleList, torch.nn.Sequential, torch.nn.ModuleDict)

In [53]:
cam = GradCAMInterpreter(model, device=device)
explanation = cam.interpret('cat.jpeg', 'head.fc', label=37)
print("Explanation generated and saved to cam_output.png")

TypeError: 'torch.device' object is not subscriptable